In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv("/mnt/c/users/ankku/Documents/WiDS-2024/moneycontrol_stock.csv")  # Replace with your file path
data

,Title,Abstract,Content
0,"Karnataka moves to tax mining land, leases; pr...",This proposed bill comes after a recent Suprem...,The Karnataka Mineral Tax Bill 2024 has propos...
1,Accumulate Avanti Feeds; target of Rs 715: Geo...,Geojit Financial Services recommended accumula...,Geojit Financial Services research report onAv...
2,Mid-day Mood | Sensex falls over 450 pts from ...,"Banking heavyweights HDFC Bank, ICICI Bank, SB...",Nifty and Sensex pulled back from their highs ...
3,Buy HDFC Bank; target of Rs 2100: Emkay Global...,Emkay Global Financial is bullish on HDFC Bank...,Emkay Global Financial's research report onHDF...
4,Buy ICICI Bank; target of Rs 1550: Sharekhan,Sharekhan is bullish on ICICI Bank has recomme...,Sharekhan's research report onICICI Bank Loan ...
...,...,...,...
600,Maharashtra Election win fires up PSU stocks –...,The BJP-led alliance crossed 200-mark out of 2...,PSU stocks hogged limelight on Dalal Street on...
601,"Agri picks report August 09, 2024: Geojit Fina...","According to Geojit, Farmers in Gujarat have s...",Geojit Financial Services's report on Daily Ag...
602,RITES surges 13% as Northeast Frontier Railway...,"In Q2, RITES bagged over 90 orders, achieving ...",Shares of RITES surged close to 13 percent in ...
603,NTPC shares rise 3% on Bernstein's 'outperform...,While Bernstein analysts don’t see significant...,Shares ofNTPCsurged 3 percent to Rs 377 in mor...


In [ ]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration
import nltk

# Download sentence tokenizer
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# Load question generation model
qa_pairs = []
for i in range(480, len(data)):
    print(i)
    model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    qg_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

    # Load question-answering model
    qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

    # Input content
    abstract = data.loc[i, 'Abstract']
    # print(abstract)

    # Split content into sentences
    sentences = sent_tokenize(abstract)

    # Generate questions and answers
    for sentence in sentences:
        # Generate question
        result = qg_pipeline(f"generate question: {sentence}")
        question = result[0]['generated_text']
        # print(question)

        # Generate answer
        answer = qa_pipeline(question=question, context=abstract)['answer']
        # print(answer)

        # Save question-answer pair
        qa_pairs.append((question, answer))

print(qa_pairs)


In [ ]:
qa_pairs

In [ ]:
qa_df = pd.DataFrame(qa_pairs, columns=["Ques", "Ans"])
qa_df["Ques"] = qa_df["Ques"].str.replace("question: ", "", regex=False)
qa_df

In [ ]:
qa_df1 = pd.DataFrame(qa_pairs, columns=["Ques", "Ans"])
qa_df1["Ques"] = qa_df1["Ques"].str.replace("question: ", "", regex=False)
qa_df1

In [ ]:
qa_df2 = pd.DataFrame(qa_pairs, columns=["Ques", "Ans"])
qa_df2["Ques"] = qa_df2["Ques"].str.replace("question: ", "", regex=False)
qa_df2

In [ ]:
qa_df_merged = pd.concat([qa_df, qa_df1])
qa_df_merged = pd.concat([qa_df_merged, qa_df2])
qa_df_merged

In [ ]:
qa_df_merged = qa_df_merged.drop_duplicates()
qa_df_merged


In [19]:
qa_df_merged.to_csv('/mnt/c/Users/ankku/Downloads/moneycontrol_stock_qa.csv', index = False)